In [1]:
source("../utils.R")
options(repr.plot.width = 10, repr.plot.height = 10)

Warning message in unit(c(t, r, b, l), unit):
“NAs introduced by coercion”


# Packaging 📦

<div class="subtitle1" id="coursename">
Techniques avancées en programmation statistique <strong>R</strong>
</div>
<div class="subtitle2" id="author">
Patrick Fournier<br>
Automne 2021<br>
Université du Québec À Montréal<br>
</div>

## Cours 5: Packaging
1. Qu'est-ce qu'un package?
2. Structure d'un package **R**
3. Documentation et namespaces

## Qu'est-ce qu'un package?

* Description complète: [Writing R Extensions](https://cran.r-project.org/doc/manuals/r-release/R-exts.html)
* Peu d'exigences:
    + Un répertoire
    + Nom du répertoire $ = $ nom du package (convention)
    + Un fichier nommé `DESCRIPTION`
    + `DESCRIPTION` contient les méta informations (nom, auteurs, description, ...)
* Un tel package minimal n'est pas d'une grande utilité

### Terminologie

* *Package*: Répertoire de fichiers ajoutant des fonctionnalités à **R**
    + *source*: L'ensemble des fichiers utilisés pour le développement d'un package
    + *bundle*: Tarball (archive) contenant le package
    + *installed*: Ce que l'on obtient en exécutant `R CMD INSTALL` sur un package
    + *binary package* 🪟: zip ou tarball contenant les fichiers d'un package installé

### Outils

* Parfaitement possible de créer un package à la main
* Package [devtools](https://devtools.r-lib.org/) simplifie le travail
    + [Référence rapide (cheat sheet)](https://raw.githubusercontent.com/rstudio/cheatsheets/master/package-development.pdf)
* Plusieurs fonctionalités de devtools sont intégrées à **R**Studio

## Structure d'un package **R**
### Squelette de **R**Studio

* `.gitignore`: Fichiers ignorés par git
* `.Rbuildignore`: Fichiers ignorés lors de la création du bundle
* `<nom du projet>.Rproj`: Fichier de configuration de **R**Studio
* `DESCRIPTION`: Métadonnées du package
* `NAMESPACE`: Symboles importés/exportés
* `man`: Documentation
* `R`: Code **R**

### `DESCRIPTION`
<div class="subtitle1">
    champs obligatoires
</div>

* `Package`: Nom du package
* `Version`: Version du package
* `License`: Licence sous laquelle votre package est distribué
* `Title`: Description courte du package
* `Description`: Description longue du package
* `Author`: Auteurs du package (nom, e-mail et [roles](https://journal.r-project.org/archive/2012-1/RJournal_2012-1_Hornik~et~al.pdf))
* `Maintainer`: Responsable (unique) du package (nom et e-mail)

Par ailleurs,
* Possible et *recommandé* de fournir un seul champ `Authors@R` en remplacement de `Author` et `Maintainer`
* [Liste complète des codes MARC (pour les roles)](https://www.loc.gov/marc/relators/relaterm.html)

[Détails](https://r-pkgs.org/description.html)

### `DESCRIPTION`
<div class="subtitle1">
    champs facultatifs usuels
</div>

* `Copyright`: Détenteur du copyright (si différent de l'auteur)
* `Date`: Date de sortie du package (yyyy-mm-dd)
* `Depends`: Dépendances *attachées* du package
    + À utiliser uniquement pour spécifier la version de **R** requise
* `Suggest`: Packages utiles dont ne dépend pas le package courrant
* `Imports`: Dépendances *non attachées* du package
* `Collate`: Ordre de chargement des fichiers **R**
    + Par défaut, ordre alphabétique
* `URL`: Liste d'URL liés au package
* `BugReports`: URL pour la soumission des rapports de bugs
* `LazyData`: Les données doivent-elles être chargées paresseusement (i.e. à la demande)?
* `ByteCompile`: Les package doit-il être byte compilé?
    + Par défaut, `true`
* `BuildVignette`: Les vignettes doivent-elles être construites?
    + Par défaut, `true`
* `NeedsCompilation` Le package nécessite-t-il une phase de compilation (yes/no)?

### `DESCRIPTION`
<div class="subtitle1">
    Recommandations
</div>

* Possibilité d'utiliser des champs arbitraires
* `Depends`ne devrait être utilisé que pour spécifier une version minimal de **R**
    + Exemple: `Depends: R(>= 4.1)`
* Les champs `Imports`, `Suggests`, `Enhances` et `Linkingto` ne devraient *jamais* être modifiés à la main $ \Rightarrow $ `usethis::use_package`
* Le champ `Authors@R` doit contenir un vecteur d'objets de classe `person`
* [Liste complète des champs utilisés par **R**](https://cran.r-project.org/doc/manuals/r-release/R-exts.html#The-DESCRIPTION-file)

## Documentation et namespace

* Le répertoire `man/` contient la documentation
* Pas nécessaire/souhaitable de la modifier par nous-mêmes
* Package *roxygen2* génère la documentation automatiquement
    + Repère les commentaires commeçant par `'` et les transforme en documentation
* Mise à jour: `devtools::document`

[Détails](https://r-pkgs.org/man.html)

### Exemple

In [2]:
#' Titre
#'
#' Le second paragraphe est toujours la description. Le premier
#' paragraphe est le titre et ne devrait pas être plus long
#' qu'une phrase. Ce paragraphe donne une description générale
#' de l'objet à documenter.
#'
#' L'ensemble des paragraphes suivants contient les détails
#' de l'objet à documenter.

### Documentation

* Il est d'usage de documenter le package lui-même
* Documenter la chaîne de caractère `"_PACKAGE"`

### Documentation
<div class="subtitle1">
    Tags Roxygen
</div>

Pour les fonctions:
* `@param name description`: Décrit un des arguments d'une fonction
* `@examples`: Code **R** valide
* `@return`: Description de la valeur de retour d'une fonction

S3 (génériques et méthodes): se documentent comme des fonctions (puisque ce sont des fonctions)

### Exemple 
<div class="subtitle1">
    Shadowing
</div>

In [3]:
address <- function() "PK-5323"
x <- 42
address()

[1] "PK-5323"

In [4]:
pryr::address(x)

[1] "0x560e0faa1e48"

In [5]:
library(pryr)


Attaching package: ‘pryr’


The following object is masked _by_ ‘.GlobalEnv’:

    address




In [6]:
address(x)

ERROR: Error in address(x): unused argument (x)


### Namespace

* La plupart des packages rendent disponible certains symboles
* Pour éviter les conflits, chaque package possède son propre *namespace*
* `library(pkg)`: *attache* `pkg`
    + Symboles exportés par `pkg` disponible de l'*environement global*
    + $ \Rightarrow $ risque de conflit
* Possibilité de *charger* un package sans l'attacher
    + Symboles: disponibles
    + Nécessité d'y référer explicitement: `::`
    
[Détails](https://r-pkgs.org/namespace.html)

### Namespace
<div class="subtitle1">
    En pratique
</div>

* Controlé par le fichier `NAMESPACE`
    + Ne devrait *Jamais* être modifié à la main
* Peut être géré à partir de commentaires roxygen2 🤔
* $ \Rightarrow $ Chaque modification doit être suivie d'un `devtools::document()` pour prendre effet

### Namespace
<div class="subtitle1">
    Importation
</div>

* 99% du temps: `Import` et `::`
    + `Import: pkg` s'assure que `pkg` est installé
    + `pkg::obj` charge automatiquement `pkg` sans l'attacher
* Possible impact sur la performance causé par `::`
* Solution: attacher `obj`
    + Roxygen: `@importFrom pkg obj`

### Namespace
<div class="subtitle1">
    Exportation
</div>

* Exportation $ \Rightarrow $ ajouter une entrée dans `NAMESPACE`
* Encore une fois, on utilise Roxygen!
* Tag: `@export`
* Détermine automatiquement si fonction "normale", méthode S3, etc.